# Wire Delay

If you take a look through the e-Lab studies, you'll see that the `WireDelay.pl` script is one of the first data transformations called in most of them.  That's because it's directly related to how the cosmic ray data is originally collected.  

Understanding what the WireDelay data transformation does requires understanding the operation and physical configuration of cosmic ray detector itself.  Of course, not all users of the e-Labs' cosmic ray data have access to a CRMD, including (most likely) you, so this is a good illustration of why it's important to communicate information about your equipment and procedures when communicating scientific observations and results to other people.  We'll take that opportunity now.

## Cosmic Ray Muon Detectors: Counters and Wires

Fig 1 is a cartoon of a Cosmic Ray Muon Detector in operation.  A muon has just passed through the four stacked panels of the detector, sending signals from each panel to the Data Acquisition board (DAQ) through the wires that connect them.

![The CRMD setup](img/DetectorVisual_556x278.png) *Fig 1: A sketch of the CRMD showing signals propagating from the detector panels to the DAQ in response to a muon event*

When taking data, we want to know the times when the muon passed through each detector panel.  However, the clock that measures these events is part of the DAQ, and so the DAQ records the time signals *when it receives them*.  The delay between the two is dictated by the wires that connect the panels to the DAQ, because it takes a certain amount of time for the signal to travel from one to the other.

Fortunately, we can figure out how long this takes, because electrical signals travel at a consistent speed through wire.  By knowing the length of the wire, we can estimate the time delay between the muon event at the detector panels and the clock reading when the DAQ receives the signal.  The `WireDelay` data transformation does just that -- it corrects the muon event times recorded in the input data by subtracting the extra time introduced by the wires.

## Using WireDelay.pl

We can execute the Perl script `WireDelay.pl` by calling the Perl interpreter from the command line along with the additional parameters that the script requires:

`$ perl ./perl/WireDelay.pl <threshIn> <outputs> <geoDir> <daqId> <fw>`

where the items in angled brackets `<>` are parameters we have to specify.  These are:

* `threshIn`:  The name of the Threshold data file we'll use as input
* `outputs`: What we want to name the output file that the script will write its results to
* `geoDir`: The directory that contains geometry files for the detectors whose data we're processing
* `daqId`: The 4-digit ID number of the CRMD DAQ that recorded this data, and
* `fw`: The version number of the firmware running on the DAQ

For example, we might want 

* `threshIn` = files/6119.2016.0104.1.thresh
* `outputs` = 6119.2016.0104.1.wd
* `geoDir` = ./geo
* `daqId` = 6119
* `fw` = 1.12

in which case the command is

`$ perl ./perl/WireDelay.pl files/6119.2016.0104.1.thresh 6119.2016.0104.1.wd ./geo 6119 1.12`

## Understanding WireDelay.pl

The input Threshold data file is `6119.2016.0104.1.thresh` in the example here.  We can use the UNIX shell command `head` to get a quick look at what it looks like before the `WireDelay` data transformation acts on it:

In [1]:
!head -10 files/6119.2016.0104.1.thresh

#$md5
#md5_hex(0)
#ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec), RISING EDGE(INT), FALLING EDGE(INT)
6119.1	2457392	0.3721863017828993	0.3721863017831598	22.50	3215689647404250	3215689647406500
6119.3	2457392	0.3721863017829138	0.3721863017831598	21.25	3215689647404375	3215689647406500
6119.2	2457392	0.3721885846820747	0.3721885846822772	17.50	3215709371653125	3215709371654875
6119.4	2457392	0.3721885846820747	0.3721885846822917	18.75	3215709371653125	3215709371655000
6119.4	2457392	0.3721901866161603	0.3721901866163773	18.75	3215723212363625	3215723212365500
6119.1	2457392	0.3721901866161748	0.3721901866164496	23.75	3215723212363750	3215723212366125
6119.1	2457392	0.3721903650327546	0.3721903650329427	16.25	3215724753883000	3215724753884625


### Applying the data transformation

Now we'll run `WireDelay` and see what changes it makes to the data.  The script `WireDelay.pl` is a Perl script designed to be executed from a UNIX shell, not from a Jupyter Notebook.  We can use the "!" trick that Jupyter gives us again, though, to call the program:

In [4]:
!perl ./perl/WireDelay.pl files/6119.2016.0104.1.thresh outputs/6119.2016.0104.1.wd ./geo 6119 1.12

The script has created the output file `outputs/6119.2016.0104.1.wd`, which we can examine the same way we did the input file earlier:

In [5]:
!head -10 outputs/6119.2016.0104.1.wd

#USING WIREDELAYS: ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec)
6119.1	2457392	0.3721978758576562	0.3721978758579167	22.50
6119.3	2457392	0.3721978758576707	0.3721978758579167	21.25
6119.2	2457392	0.3722001587568317	0.3722001587570342	17.50
6119.4	2457392	0.3722001587568317	0.3722001587570486	18.75
6119.4	2457392	0.3722017606909173	0.3722017606911343	18.75
6119.1	2457392	0.3722017606909317	0.3722017606912065	23.75
6119.1	2457392	0.3722019391075115	0.3722019391076997	16.25
6119.3	2457392	0.3722019391075405	0.3722019391077576	18.75
6119.2	2457392	0.3722025586529688	0.3722025586532147	21.25


So, what has `WireDelay.pl` script done to our data?  Look for differences between this file, `6119.2016.0104.1.wd`, and the input file `6119.2016.0104.1.thresh` shown above.

#### A closer look
First, we can see that the data appears to be in the same order.  The counters are recorded in the same order `6119.1, 6119.3, 6119.2, ...` and with the same "time over threshold" values `22.50, 21.25, 17.50, ...` in both files.  If you're not convinced, increase the number of lines returned by the `head` command until the pattern is clear.

Next, we note that `WireDelay` keeps the Julian day value from the Threshold file, but it drops the raw-integer rising and falling edge values.  Evidently we won't be needing these values for further analysis.

Last, we notice the most interesting thing: the rising and falling edge time values have changed!

Investigating further, we can see that for each line of data, the output file `6119.2016.0104.1.wd` has values that are exactly `0.0000115740747569` greater than the corresponding value in the input file `6119.2016.0104.1.thresh`.  This time represents the *wire delay*: After a counter registers a hit, it takes a small amount of time for that signal to travel through the connector cable to the DAQ board, where it's recorded as data. The `WireDelay.pl` script adjusts the data for this, helping us make time comparisons between detectors as precisely as possible.

**Question 1** Why are the corrected times *greater than* the original times?  Should the WireDelay correction *subtract* the extra time the signals spent travelling through the wires?

This isn't an exercise, I'm really asking.

#### Follow the data

The obvious next question is, "Why `0.0000115740747569`?"  That is, how does `WireDelay.pl` know how much to adjust the rising and falling edge times by?  The answer is in the other input file, `6119.geo`.  We could examine it to see what it contains, but it's more fun to try to guess first.

### Exercise 1

Remember that the Rising Edge and Falling Edge numbers are given in days, not in seconds or nanoseconds.  That is, a value of `0.5000000000000000` represents half a day, or 12 hours (or 720 minutes, etc.).  This isn't the most natural way to think of time when dealing with such fast signals, so it's useful to be able to convert these into something more intuitive, like seconds or nanoseconds.

**A)** Write a Python function to convert a time value given in days into nanoseconds.  You may name it whatever you like, but I'll suggest `daysToNanosec()`.

In [ ]:
# Write your function here
# Hint: If there are 24 hours in a day, what fraction of a day is one hour?
# If there are X nanoseconds in a day, what fraction of a day is one nanosecond?
# What is X?
def daysToNanosec():
    ### Instructor's copy:
    ### 1 day = (24*60*60) sec = (24*60*60) e9 ns
    
    
    

**B)** Use the function you just wrote to convert `0.0000115740747569` days (what `WireDelay.pl` added to our data) into nanoseconds.

In [ ]:
# Write your Python code here
### Instructor's copy: the answer is 1.00000005899616 seconds, or 1.00000005899616 e9 nanoseconds




**C)** Using the rule of thumb that signals travel about 2/3 ft/ns through cabling, estimate what length of cable the experimenter used when taking this cosmic ray data.  Do you think your answer is reasonable?  

In [ ]:
### Instructor's copy: 0.66666670599744 e9 feet, or ~126,263 miles.
### This is just between the equatorial circumference of Saturn (117,649 mi) and that of Jupiter (139,559 mi)
### The moon's orbital radius is about twice this value, at 239,000 miles.
### This does not sound reasonable.





## Now try it with Parsl!

To use the Parslized scripts, we'll first prepare Parsl:

In [ ]:
import parsl
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from parsl.app.app import bash_app,python_app
from parsl import File

config = Config(
    executors=[ThreadPoolExecutor()],
    lazy_errors=True
)
parsl.load(config)

We write a Python function to return the shell command we saw above, and then we turn that function into a Parsl App with the `@App` decorator:

In [ ]:
@App('bash', dfk)
def WireDelay(threshIn='', outputs=[], geoDir='', daqId='', fw='', stdout='stdout.txt', stderr='stderr.txt'):
        return 'perl ./perl/WireDelay.pl %s %s %s %s %s' %(threshIn,outputs[0],geoDir,daqId,fw)